Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.
В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

## Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [5]:
import scipy
from statsmodels.stats.weightstats import *

In [6]:
data_normal = data[data['Diagnosis'] =='normal']

In [7]:
data_cancer= data[data['Diagnosis']=='cancer']
data_early_neoplasia=data[data['Diagnosis']=='early neoplasia']

In [8]:
normal_cancer = []
##for i,name in enumerate(data.columns):
##  if i > 1:
##        continue
    
##    p_value = scipy.stats.ttest_ind(np.array(data_normal[name]), np.array(data_cancer[name]), equal_var = False)[1]
##    normal_cancer.append((name, p_value))

In [9]:
normal=np.array((data_normal.drop('Diagnosis', axis=1)).drop('Patient_id',axis=1))
early_neoplasia=np.array((data_early_neoplasia.drop('Diagnosis', axis=1)).drop('Patient_id',axis=1))

In [10]:
cancer =np.array((data_cancer.drop('Diagnosis', axis=1)).drop('Patient_id',axis=1))

In [11]:
cancer_early_neoplasia = []
for i in range(len(data.columns)-2):
    statistic, p = scipy.stats.ttest_ind(cancer[:,i], early_neoplasia[:,i], equal_var = False)
    cancer_early_neoplasia.append([data.columns[i+2], statistic,p])

In [12]:
for i in range(len(data.columns)-2):
    statistic, p = scipy.stats.ttest_ind(normal[:,i], early_neoplasia[:,i], equal_var = False)
    normal_cancer.append([data.columns[i+2], statistic,p])

In [13]:
n_e = pd.DataFrame.from_records(normal_cancer)
n_e.columns = ['gene','statistic','p-value']

In [14]:
c_e = pd.DataFrame.from_records(cancer_early_neoplasia)
c_e.columns = ['gene_1','statistic_1','p-value_1']

In [15]:
n_e[n_e['p-value']<0.05].shape

(1575, 3)

In [16]:
f = open('task11.txt', 'w')
f.write(str(n_e[n_e['p-value']<0.05].shape[0]))
f.close()

In [17]:
f = open('task12.txt', 'w')
f.write(str(c_e[c_e['p-value_1']<0.05].shape[0]))
f.close()

## Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

In [18]:
import statsmodels.stats.multitest as smm

Поправка Холма для первого набора:

In [19]:
reject_n_e, p_corrected_n_e, a1, a2 = smm.multipletests(n_e['p-value'], 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [20]:
n_e['p_corrected_n_e'] = p_corrected_n_e
n_e['reject_n_e'] = reject_n_e

In [21]:
n_e[n_e['reject_n_e'] == True].shape

(2, 5)

In [22]:
reject_c_e, p_corrected_c_e, b1, b2 = smm.multipletests(c_e['p-value_1'], 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [23]:
c_e['p_corrected_c_e'] = p_corrected_c_e
c_e['reject_c_e'] = reject_c_e

In [24]:
c_e[c_e['reject_c_e'] == True].shape

(79, 5)

In [25]:
practical = []
for i in range(15748):
    if normal.mean(axis = 0)[i]<early_neoplasia.mean(axis = 0)[i]:
        F = early_neoplasia.mean(axis = 0)[i]/float(normal.mean(axis = 0)[i])
    if normal.mean(axis = 0)[i]>early_neoplasia.mean(axis = 0)[i]:
        F = -float(normal.mean(axis = 0)[i])/early_neoplasia.mean(axis = 0)[i]
    practical.append(F)

In [84]:
practical_1 = []
for i in range(15748):
    if early_neoplasia.mean(axis = 0)[i]<cancer.mean(axis = 0)[i]:
        F = cancer.mean(axis = 0)[i]/float(early_neoplasia.mean(axis = 0)[i])
    if early_neoplasia.mean(axis = 0)[i]>cancer.mean(axis = 0)[i]:
        F = -float(early_neoplasia.mean(axis = 0)[i])/cancer.mean(axis = 0)[i]
    practical_1.append(F)

In [27]:
n_e['practical'] = practical

In [43]:
f = open('task21.txt', 'w')
f.write(str(n_e[n_e['reject_n_e'] == True][abs(n_e['practical'])>1.5].shape[0]))
f.close()

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [85]:
c_e['practical_1'] = practical_1

In [88]:
c_e[abs(c_e['practical_1'])>=1.5][c_e['p_corrected_c_e'] < 0.025].shape

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(77, 8)

In [89]:
f = open('task22.txt', 'w')
f.write(str(c_e[abs(c_e['practical_1'])>=1.5][c_e['p_corrected_c_e'] < 0.025].shape[0]))
f.close()

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


## Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0H 
0
​	 , когда они есть, и будут чаще отклонять H_0H 
0
​	 , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [48]:
reject_n_e_1, p_corrected_n_e_1, a1, a2 = smm.multipletests(n_e['p-value'], 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [49]:
n_e['p_corrected_n_e_1'] = p_corrected_n_e_1
n_e['reject_n_e_1'] = reject_n_e_1

In [50]:
n_e[n_e['reject_n_e_1'] == True][abs(n_e['practical'])>1.5].shape[0]

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


4

In [69]:
reject_c_e_1, p_corrected_c_e_1, b1, b2 = smm.multipletests(c_e['p-value_1'], 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [70]:
c_e['p_corrected_c_e_1'] = p_corrected_c_e_1
c_e['reject_c_e_1'] = reject_c_e_1

In [90]:
c_e[abs(c_e['practical_1'])>=1.5][c_e['p_corrected_c_e_1'] < 0.025].shape

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(524, 8)

In [73]:
f = open('task31.txt', 'w')
f.write(str(n_e[n_e['reject_n_e_1'] == True][abs(n_e['practical'])>1.5].shape[0]))
f.close()

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [91]:
f = open('task32.txt', 'w')
f.write(str(c_e[abs(c_e['practical_1'])>=1.5][c_e['p_corrected_c_e_1'] < 0.025].shape[0]))
f.close()

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
